# Линейная регрессия

На сегодняшнем занятии будем работать с датасетом BlackFriday (ниже есть его описание):

https://www.kaggle.com/mehdidag/black-friday/version/1

Description
The dataset here is a sample of the transactions made in a retail store. The store wants to know better the customer purchase behaviour against different products. Specifically, here the problem is a regression problem where we are trying to predict the dependent variable (the amount of purchase) with the help of the information contained in the other variables.

Classification problem can also be settled in this dataset since several variables are categorical, and some other approaches could be "Predicting the age of the consumer" or even "Predict the category of goods bought". This dataset is also particularly convenient for clustering and maybe find different clusters of consumers within it.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%pylab inline

In [ ]:
data = pd.read_csv('BlackFriday.csv')
data.head()

# Подготовка данных

In [ ]:
data.shape

In [ ]:
missing_values = data.isnull().sum().sort_values(ascending = False)
missing_values.head()

Скорее всего NaN-ы в Product_Category_2 и Product_Category_3 означают, что человек не покупал продукты из этих категорий.

Поэтому можем заменить эти пропущенные значения на 0.

In [ ]:
data = data.fillna(0)

Посмотрим, какие типы данных есть в нашем датасете. 

Выведите на экран тип каждой колонки в таблице.

In [ ]:
#your code here

Обработаем нечисловые колонки.

1) Выведите на экран все возможные значения из столбца Gender.

2) Замените в столбце Gender значение 'M' на 1, а 'F' на 0.

In [ ]:
#your code here

Поработаем со столбцом Age.

1) Выведите на экран все возможные значения из столбца Age.

2) Напишите функцию для перевода каждого диапазона возрастов в число (самые маленькие возрасты - 0, следующий диапазон - 1 и т.д.). Замените значения в столбце Age на числовые, применив эту функцию.

In [ ]:
data['Age'].unique()

def map_age(age):
    if age == '0-17':
        return 0
    elif age == '18-25':
        #your code here
        
data['Age'] = data['Age'].apply(map_age)

Обработаем столбец City_Category.

1) Выведите все уникальные значения данного столбца.

2) Закодируйте столбец с помощью OneHotEncoding (get_dummies).

In [ ]:
#your code here
data['City_Category'].unique()

data = pd.get_dummies(data, columns=['City_Category'], dtype=int)
data.head()

Наконец, обработаем Stay_In_Current_City_Years.

1) Выведите на экран все уникальные значения данного столбца.

2) Замените '4+' на 4, а все остальные значения оставьте как есть, но переведите их в числовой вид (x -> int(x)).

In [ ]:
data['Stay_In_Current_City_Years'].unique()

def map_stay(years):
    #your code here
    
data['Stay_In_Current_City_Years'] = #your code here

In [ ]:
data.head()

Удалим столбцы User_ID и Product_ID, так как они не несут полезной для нас информации.

In [ ]:
#your code here

In [ ]:
data.head()

# Исследование данных

1. Как влияет пол на целевую переменную Purchase? Постройте диаграмму (barplot).

2. Как влияет возраст на целевую переменную Purchase? Постройте диаграмму (barplot).

3. Как влияет City_Category на целевую переменную Purchase? Постройте диаграмму (barplot).

In [ ]:
#your code here

Нарисуйте матрицу корреляций признаков.

In [ ]:
plt.figure(figsize=(12,9))
#your code here

## Масштабирование (нормализация) признаков в линейных моделях

В линейных и метрических (основанных на вычислении расстояния между объектами) моделях необходимо масштабировать признаки.

***Нормализация признаков*** - приведение всех признаков к одинаковому масштабу (например, приведение всех значений на отрезок $[0;1]$).

***Зачем нужна нормализация?*** Если машстабы измерений признаков существенно (на несколько порядков) различаются, то появляется опасноcть, что будут учитываться только «крупномасштабные» признаки.

***Как делать нормализацию в Python:*** StandardScaler или MinMaxScaler из библиотеки sklearn.preprocessing.

# Построение модели

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

X = data.copy()

y = data['Purchase']
X.drop(['Purchase'], axis=1, inplace=True)

indices = np.arange(len(X))

Xtrain, Xtest, ytrain, ytest, idx_train, idx_test = train_test_split(
                                                    X, y, indices, test_size=0.2, random_state=111)

scaler = StandardScaler().fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)

regressor = LinearRegression()
regressor.fit(Xtrain, ytrain)

ypred_train = regressor.predict(Xtrain)
ypred_test = regressor.predict(Xtest)

r2_score(ytrain,ypred_train), r2_score(ytest,ypred_test)

***Посмотрим на веса обученной модели.***

In [ ]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(regressor.coef_))], axis = 1)
coefficients

Добавим L1 регуляризацию.

In [ ]:
from sklearn.linear_model import Lasso

regressor2 = Lasso(alpha=10)
regressor2.fit(Xtrain, ytrain)

ypred2 = regressor2.predict(Xtest)

print('Scores:', r2_score(ytest,ypred2), mean_squared_error(ytest,ypred2))
print(regressor2.coef_,'\n')

Посмотрим, что происходит при разных значениях параметра регуляризации $\alpha$

In [ ]:
from sklearn.linear_model import Lasso

for a in np.arange(0.1,100.1,25):
    regressor2 = Lasso(alpha=a)
    regressor2.fit(Xtrain, ytrain)

    ypred2 = regressor2.predict(Xtest)

    print('alpha={}'.format(a))
    print('Scores:', r2_score(ytest,ypred2), mean_squared_error(ytest,ypred2))
    print(regressor2.coef_,'\n')

**Видим, что L1-регуляризация зануляет некоторые веса и таким образом производит отбор признаков.**

Подберем оптимальное значение параметра регуляризации по кросс-валидации

In [ ]:
from sklearn.linear_model import LassoCV

n_alphas = 200
alphas = np.linspace(0.1, 10, n_alphas)

lasso_cv = LassoCV(alphas=alphas, 
                   cv=3, random_state=17)
lasso_cv.fit(X, y)

In [ ]:
lasso_cv.coef_

In [ ]:
lasso_cv.alpha_

Обучим модель с найденным параметром регуляризации $\alpha$

In [ ]:
regressor2 = Lasso(alpha=lasso_cv.alpha_)

regressor2.fit(Xtrain, ytrain)

ypred2_train = regressor2.predict(Xtrain)
ypred2_test = regressor2.predict(Xtest)

print('Scores:', r2_score(ytrain,ypred2_train), r2_score(ytest,ypred2_test))

Добавим L2 регуляризацию.

In [ ]:
from sklearn.linear_model import Ridge

for a in np.arange(-10,10.1,2.5):
    regressor3 = Ridge(alpha=a)
    regressor3.fit(Xtrain, ytrain)

    ypred3 = regressor3.predict(Xtest)

    print('alpha={}'.format(a))
    print('Scores:',r2_score(ytest,ypred3), mean_squared_error(ytest,ypred3))
    print(regressor3.coef_)

**Ridge регрессия не зануляет веса (они могут быть очень маленькими, но ненулевыми).**

Теперь подберем оптимальное a по кросс-валидации.

In [ ]:
from sklearn.linear_model import RidgeCV

n_alphas = 200
ridge_alphas = np.logspace(-2, 6, n_alphas)

ridge_cv = RidgeCV(alphas=ridge_alphas, 
                   scoring='neg_mean_squared_error',
                   cv=3)
ridge_cv.fit(X, y)

In [ ]:
ridge_cv.alpha_

In [ ]:
regressor3 = Lasso(alpha=ridge_cv.alpha_)

regressor3.fit(Xtrain, ytrain)

ypred3_train = regressor3.predict(Xtrain)
ypred3_test = regressor3.predict(Xtest)

print('Scores:', r2_score(ytrain,ypred3_train), r2_score(ytest,ypred3_test))

Добавим одновременно и L1, и L2 регуляризацию.

У ElasticNet два параметра: alpha и l1_ratio. Используйте ElasticNet со значениями параметров alpha и l1_ratio равными 0.5.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [ ]:
from sklearn.linear_model import ElasticNet

#your code here

# Feature Engineering

Попробуйте соорудить новые признаки.

Идеи:

1) Удаление зависимых признаков.

2) Кодирование категориальных признаков: OHE / счётчики / вручную.

3) Перемножение признаков, которые в паре могут влиять на результат (линейная регрессия не умеет перемножать признаки)

4) Взятие других функций от признаков и другие идеи.

#### Шаг 1. Пример.

Как вы думаете, какие признаки из представленных явно зависят между собой?

In [ ]:
#your code here

#удаляем Product_Category_C

#### Шаг 2. Пример создания счётчика.

In [ ]:
def code_mean(data, cat_feature, real_feature):
    """
    Возвращает словарь, где ключами являются уникальные категории признака cat_feature, 
    а значениями - средние по real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

In [ ]:
data['occupation_counter'] = list(map(code_mean(data.iloc[idx_train], 'Occupation', "Purchase").get, data.Occupation))
data[['Occupation','occupation_counter']].head()

#### Шаг 3.

Посмотрим на матрицу корреляций и исходя из неё придумаем новые признаки.

Идея: попробуйте перемножить Product_Category_1 и Occupation.

In [ ]:
X['Pr1_Occupation'] = #your code here

#### Шаг 4.
Опять же, возьмём идеи из матрицы корреляций.

Идея: попробуйте возвести Product_Category_1 в степень (2, 3 и т.д.), взять корень или логарифм из Product_Category_3.

In [ ]:
X['Pr1_sq'] = #your code here
X['Pr3_log'] = #your code here